## Финальное соревнование

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

В данном задании вас ждет неизвестная зависимость. Ваша основная задача: **построить две лучших модели**, минимизирующих среднеквадратичную ошибку (MSE):
1. На первую модель не налагается ограничений.
2. Вторая модель должна быть **линейной**, т.е. представлять собой линейную комбинацию признаков плюс свободный член: $\boldsymbol{w}^{\top}\boldsymbol{x} + b$. При этом __вы можете использовать базовые математические операции для преобразования признаков__: np.exp, np.log, np.pow (полный список доступен в [документации](https://numpy.org/doc/stable/reference/routines.math.html)), а также линейные операции над ними (сумма, умножение на число и пр.). Для преобразования признаков вам будет необходимо написать функцию `my_transformation`. __Кол-во параметров (весов) используемых второй моделью не должно превышать 15 (включая свободный член).__

Настоятельно рекомендуем написать код "с нуля", лишь поглядывая на готовые примеры, а не просто "скопировать-вставить". Это поможет вам в будущем писать код более уверенно

In [ ]:
import os
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns

Загрузка данных происходит ниже. Если она не срабатывает, самостоятельно скачайте файл `hw_final_open_data.npy` и положите его в ту же директорию, что и ноутбук.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy -O hw_final_open_data.npy
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy -O hw_final_open_target.npy

--2023-12-03 10:37:19--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44928 (44K) [application/octet-stream]
Saving to: ‘hw_final_open_data.npy’

hw_final_open_data. 100%[===================>]  43.88K  --.-KB/s    in 0.04s   

2023-12-03 10:37:19 (1.02 MB/s) - ‘hw_final_open_data.npy’ saved [44928/44928]

--2023-12-03 10:37:19--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubus

In [ ]:
assert os.path.exists('hw_final_open_data.npy'), 'Please, download `hw_final_open_data.npy` and place it in the working directory'
assert os.path.exists('hw_final_open_target.npy'), 'Please, download `hw_final_open_target.npy` and place it in the working directory'
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

Разбивка на `train` и `val` опциональна и сделана для вашего удобства.

In [ ]:
df_data = pd.DataFrame(np.c_[data, target], columns = ['0', '1', '2', '3', '4', '5', '6', 'y'])
df_data.head(2)

,0,1,2,3,4,5,6,y
0,-0.565814,-0.430544,-0.56863,-0.379486,-0.527420,-0.160691,-0.575438,0.382870
1,-0.562880,-0.051177,-0.56863,-0.134094,-0.506339,0.795601,-0.575438,2.742402


In [ ]:
def cutt_off_std_column(data, column, cut_off_k = 3):
  outliers_mean, outliers_std = np.mean(data[column]), np.std(data[column])
  cut_off = outliers_std * cut_off_k
  lower, upper = outliers_mean - cut_off, outliers_mean + cut_off

  return data[(data[column] > lower) & (data[column] < upper)]


In [ ]:
df_data = cutt_off_std_column(df_data, '6', cut_off_k = 1)
df_data = cutt_off_std_column(df_data, '2', cut_off_k = 1)
df_data.describe()

,0,1,2,3,4,5,6,y
count,774.000000,774.000000,7.740000e+02,774.000000,774.000000,774.000000,7.740000e+02,774.000000
mean,-0.562254,0.283531,-5.686305e-01,0.013925,-0.499418,1.581590,-5.754376e-01,3.820436
std,0.001769,0.504701,4.709064e-15,0.249139,0.015170,1.206140,9.233220e-14,2.146101
min,-0.566620,-0.474994,-5.686305e-01,-0.417683,-0.532201,-0.279446,-5.754376e-01,-0.215767
25%,-0.563475,-0.169432,-5.686305e-01,-0.201865,-0.511150,0.504257,-5.754376e-01,2.006795
50%,-0.561872,0.230017,-5.686305e-01,0.011340,-0.497413,1.474881,-5.754376e-01,3.716754
75%,-0.560765,0.707639,-5.686305e-01,0.228319,-0.486367,2.600914,-5.754376e-01,5.619954
max,-0.559918,1.250293,-5.686305e-01,0.448173,-0.476899,3.853662,-5.754376e-01,8.013126


### Ridge

In [ ]:
def train_prepare_data(npdata, target, rs = 42):
  df_data = pd.DataFrame(np.c_[npdata, target], columns = ['0', '1', '2', '3', '4', '5', '6', 'y'])

  df_data = cutt_off_std_column(df_data, '6', cut_off_k = 1)
  df_data = cutt_off_std_column(df_data, '2', cut_off_k = 1)

  # df_data = pd.DataFrame(ct.fit_transform(df_data), columns = ['0', '1', '2', '3', '4', '5', '6', 'y'])

  df_data.drop(columns=['2', '6'], inplace=True)
  # df_data.drop(columns=['2', '3', '4', '5', '6'], inplace=True)

  train_x, valid_x, train_y, valid_y = train_test_split(df_data.drop(columns=['y']), df_data['y'], test_size=0.2, random_state = rs)

  return train_x, valid_x, train_y, valid_y

def valid_prepare_data(npdata):
  df_data = pd.DataFrame(npdata, columns = ['0', '1', '2', '3', '4', '5', '6'])

  # df_data = pd.DataFrame(ct.fit_transform(df_data), columns = ['0', '1', '2', '3', '4', '5', '6'])

  df_data.drop(columns=['2', '6'], inplace=True)
  # df_data.drop(columns=['2', '3', '4', '5', '6'], inplace=True)

  return df_data

In [ ]:
def show_scores(model, X, y, dataset = 'valid'):
  print(
    f'{dataset} mse =\t {mean_squared_error(np.round(model.predict(X), 2), np.round(y)):.5f}',
    sep='\n'
  )

In [ ]:
from sklearn.linear_model import Ridge

train_x, valid_x, train_y, valid_y = train_prepare_data(data, target, rs = 12412412)

baseline_model = Ridge(random_state=1)
baseline_model.fit(train_x, train_y)

show_scores(baseline_model, train_x, train_y, 'train')
show_scores(baseline_model, valid_x, valid_y, 'valid')

# 1.36
# train mse =	 0.18599
# valid mse =	 0.15940

# train mse =	 0.18814
# valid mse =	 0.15281

train mse =	 0.18041
valid mse =	 0.17525


In [ ]:
def train_prepare_data(npdata, target, rs = 42):
  df_data = pd.DataFrame(np.c_[npdata, target], columns = ['0', '1', '2', '3', '4', '5', '6', 'y'])

  df_data = cutt_off_std_column(df_data, '6', cut_off_k = 1)
  df_data = cutt_off_std_column(df_data, '2', cut_off_k = 1)

  pca = PCA(n_components=2).fit(train_x_[['1', '3', '5']].to_numpy())
  df_data[['123', '123_1']] = pca.transform(df_data[['1', '3', '5']].to_numpy())
  df_data.drop(columns = ['1', '3', '5', '123_1'], inplace=True)

  pca_2 = PCA(n_components=1).fit(df_data[['0', '4']].to_numpy())
  df_data[['04']] = pca_2.transform(df_data[['0', '4']].to_numpy())
  df_data.drop(columns = ['0', '4'], inplace=True)

  train_x, valid_x, train_y, valid_y = train_test_split(df_data.drop(columns=['y']), df_data['y'], test_size=0.17, random_state = rs)

  return pca, train_x, valid_x, train_y, valid_y

def valid_prepare_data(npdata, ct):
  df_data = pd.DataFrame(npdata, columns = ['0', '1', '2', '3', '4', '5', '6'])

  # df_data.drop(columns=['2', '6'], inplace=True)

  return df_data

In [ ]:
from sklearn.decomposition import PCA
train_x_ = train_x

pca = PCA(n_components=2).fit(train_x_[['1', '3', '5']].to_numpy())
train_x_[['123', '123_1']] = pca.transform(train_x_[['1', '3', '5']].to_numpy())
train_x_.drop(columns = ['1', '3', '5'], inplace=True)

pca_2 = PCA(n_components=1).fit(train_x_[['0', '4']].to_numpy())
train_x_[['04']] = pca_2.transform(train_x_[['0', '4']].to_numpy())
train_x_.drop(columns = ['0', '4'], inplace=True)
train_x_.corr()

KeyError: ignored

In [ ]:
train_x_with_y = train_x_
train_x_with_y['y'] = train_y

In [ ]:
sns.pairplot(train_x_with_y)

In [ ]:
train_x_ = train_x_with_y.drop(columns=['y'])
train_y_ = train_x_with_y['y']

baseline_model_2 = Ridge(random_state=13)
baseline_model_2.fit(train_x_, train_y_)

show_scores(baseline_model_2, train_x_, train_y, 'train')
# show_scores(baseline_model, valid_x, valid_y, 'valid')

### Модель №1
Напоминаем, в первой части задания ваша основная задача – получить наилучший результат без ограничений на модель. Сдаваться будут только предсказания модели.

Пример с использованием Random Forest доступен ниже.

In [ ]:
rf = RandomForestRegressor()
rf.fit(train_x_with_y.drop(columns=['y']), train_x_with_y['y'])

print(
    f'train mse =\t {mean_squared_error(np.round(rf.predict(train_x), 2), np.round(train_y)):.5f}',
    f'validation mse = {mean_squared_error(np.round(rf.predict(valid_x)), np.round(valid_y)):.5f}',
    sep='\n'
)

##### Сдача первой части соревнования
Загрузите файл `hw_final_closed_data.npy` (ссылка есть на странице с заданием). Если вы используете sklearn-совместимую модель, для генерации посылки вы можете воспользоваться функцией `get_predictions`. В ином случае перепишите функцию для вашей модели и запустите код под следующей ячейкой для генерации посылки.

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy -O hw_final_closed_data.npy

--2023-12-03 10:33:24--  https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11328 (11K) [application/octet-stream]
Saving to: ‘hw_final_closed_data.npy’

hw_final_closed_dat 100%[===================>]  11.06K  --.-KB/s    in 0.001s  

2023-12-03 10:33:24 (14.2 MB/s) - ‘hw_final_closed_data.npy’ saved [11328/11328]



## Output 1

In [ ]:
assert os.path.exists('hw_final_closed_data.npy'), 'Please, download `hw_final_closed_data.npy` and place it in the working directory'
closed_data = np.load('hw_final_closed_data.npy', allow_pickle=False)

Если необходимо, преобразуйте данные. Преобразованную матрицу объект-признак сохраните в переменную `closed_data`.

In [ ]:
# optional transformations

In [ ]:
def get_predictions(model, eval_data, step=10):
    predicted_values = model.predict(eval_data)
    return predicted_values

Обращаем ваше внимание, предсказания округляются до сотых!

In [ ]:
eval_df = valid_prepare_data(closed_data)
eval_df

,0,1,3,4,5
0,-0.559878,1.281356,0.460149,-0.476420,3.924734
1,-0.561259,0.467296,0.122720,-0.491478,2.037739
2,-0.564778,-0.345467,-0.315276,-0.520689,0.060612
3,-0.559726,1.401775,0.506055,-0.474613,4.199694
4,-0.559901,1.263218,0.453163,-0.476699,3.883242
...,...,...,...,...,...
195,-0.562022,0.180290,-0.013141,-0.498809,1.355861
196,-0.566792,-0.482617,-0.424728,-0.533172,-0.300128
197,-0.560755,0.712904,0.230566,-0.486263,2.613184
198,-0.560232,1.027075,0.360342,-0.480519,3.341058


In [ ]:
# predicted_values = np.round(get_predictions(model=rf, eval_data=closed_data), 2)
predicted_values = np.round(get_predictions(model=baseline_model, eval_data=eval_df), 2)

assert predicted_values.shape == (closed_data.shape[0], ) # predictions should be just one-dimensional array

In [ ]:
# do not change the code in the block below
# __________start of block__________
def float_list_to_comma_separated_str(_list):
    _list = list(np.round(np.array(_list), 2))
    return ','.join([str(x) for x in _list])

submission_dict = {
    'predictions': float_list_to_comma_separated_str(predicted_values)
}
with open('submission_dict_final_p01.json', 'w') as iofile:
    json.dump(submission_dict, iofile)

print('File saved to `submission_dict_final_p01.npy`')
# __________end of block__________

File saved to `submission_dict_final_p01.npy`


### Модель №2
Функция `my_transformation` принимает на вход матрицу объект-признак (`numpy.ndarray` типа `np.float`) и преобразует ее в новую матрицу. Данная функция может использовать только numpy-операции, а также арифметические действия.

Для примера доступна функция ниже. Она лишь добавляет новый признак, представляющий собой произведение первого и второго исходных признаков (считая с нуля).

In [ ]:
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0] * feature_matrix[:, 1]
    return new_feature_matrix

In [ ]:
transformed_train_x = my_transformation(train_x)

In [ ]:
lr = Ridge()
lr.fit(transformed_train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(lr.predict(transformed_train_x), train_y):.5f}',
    f'validation mse = {mean_squared_error(lr.predict(my_transformation(valid_x)), valid_y):.5f}',
    sep='\n'
)

Обращаем ваше внимание, что параметры линейной модели будут округляться до __четырех знаков после запятой__. Это не должно сильно повлиять на качество предсказаний:

In [ ]:
original_predictions = lr.predict(transformed_train_x)
rounded_predictions = transformed_train_x.dot(np.round(lr.coef_, 4)) + np.round(lr.intercept_, 4)


assert np.allclose(original_predictions, rounded_predictions, atol=1e-3)

Параметры вашей модели:

In [ ]:
w_list = list(np.round(lr.coef_, 4))
print(f'w = {list(np.round(lr.coef_, 4))}\nb = {np.round(lr.intercept_, 4)}')

Напоминаем, ваша модель не должна использовать более 15 параметров (14 весов плюс свободный член).

In [ ]:
assert len(w_list) + 1 <= 15

##### Сдача второй части соревнования
Для сдачи вам достаточно отправить функцию `my_transformation` и параметры вашей модели в контест в задачу №2. Пример посылки доступен ниже. Имортирование `numpy` также необходимо.

In [ ]:
# __________example_submission_start__________
import numpy as np
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0
    ] * feature_matrix[:, 1]
    return new_feature_matrix

w_submission = [-0.0027, -0.2637, 0.0, -0.1134, -0.0165, -0.9329, 0.0, 0.1293]
b_submission = 1.1312
# __________example_submission_end__________

На этом задание завершено. Поздравляем!